this notebook analyzes https://github.com/leelabcnbc/thesis-yimeng-v2/blob/d76feb9e22bf751935f48dd13571286cc31fd6e5/scripts/feature_extraction/yuanyuan_8k_a/maskcnn_polished_with_rcnn_k_bl/submit_20200523.py

In [1]:
from os.path import dirname, relpath, realpath, join, exists

import h5py

from numpy.linalg import norm
import numpy as np

from thesis_v2 import dir_dict
from thesis_v2.submission import utils
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200523_8k_feature_extraction_generator,
    script_keygen,
    gen_feature_extraction_global_vars
)
from json import dump, load

In [2]:
def generate_diff_vec(*, tensor_list, acc_mode, max_len):
    assert len(tensor_list) > 1
    
    if acc_mode=='instant':
        pass
    elif acc_mode == 'cummean':
        tensor_list_new = []
        for i in range(len(tensor_list)):
            tensor_list_new.append(np.mean(np.asarray(tensor_list[:i+1]), axis=0))
        tensor_list = tensor_list_new
    else:
        raise ValueError
    
    tensor_shape = tensor_list[0].shape
    # get norm diff
    diff_vec = np.full((max_len,), fill_value=np.nan, dtype=np.float64)
    
    for i in range(len(tensor_list)-1):
        vec_prev = tensor_list[i]
        vec_now = tensor_list[i+1]
        diff_this = norm(vec_now.ravel()-vec_prev.ravel())/norm(vec_prev.ravel())
        diff_vec[i] = diff_this
    return diff_vec.tolist()

def load_all():
    
    batch_key='yuanyuan_8k_a/20200523'
    global_vars_for_feature_extraction = gen_feature_extraction_global_vars(key=batch_key)
    
    for idx, param_dict in enumerate(explored_models_20200523_8k_feature_extraction_generator()):
        
        cls = param_dict['rcnn_bl_cls']
        if cls <= 1:
            continue
        
        key_this = script_keygen(**param_dict)        
        h5file = join(global_vars_for_feature_extraction['feature_file_dir'], key_this+'.hdf5')
        
        auxfile = join(global_vars_for_feature_extraction['feature_file_dir'], key_this+'.json')
        
        if not exists(auxfile):
            with h5py.File(h5file, 'r') as f:
                g = f['test']
                resp_neurons = [g[f'1.{x}'][()] for x in range(cls)]
                diff_vec_neurons = generate_diff_vec(
                    tensor_list = resp_neurons,
                    acc_mode='instant',
                    max_len = 6,
                )

                resp_map = [g[f'0.{x}'][()] for x in range(cls)]
                diff_vec_map_instant = generate_diff_vec(
                    tensor_list = resp_map,
                    acc_mode='instant',
                    max_len = 6,
                )

                diff_vec_map_cummean = generate_diff_vec(
                    tensor_list = resp_map,
                    acc_mode='cummean',
                    max_len = 6,
                )
            # write aux file.
            with open(auxfile, 'wt', encoding='utf-8') as f_aux:
                dump({
                    'test': {
                        'diff_final_act': diff_vec_neurons,
                        'diff_bl_stack_instant': diff_vec_map_instant,
                        'diff_bl_stack_cummean': diff_vec_map_cummean,
                    }
                },
                    f_aux,
                )
        with open(auxfile, 'rt', encoding='utf-8') as f_aux:
            aux_obj = load(f_aux)
        
        if idx % 10 == 0:
            print(idx)
            print(aux_obj)
load_all()

10
{'test': {'diff_final_act': [0.053509004414081573, nan, nan, nan, nan, nan], 'diff_bl_stack_instant': [0.4511548578739166, nan, nan, nan, nan, nan], 'diff_bl_stack_cummean': [0.2255774289369583, nan, nan, nan, nan, nan]}}
20
{'test': {'diff_final_act': [0.10487790405750275, 0.05069964379072189, nan, nan, nan, nan], 'diff_bl_stack_instant': [0.9095392227172852, 0.5398644804954529, nan, nan, nan, nan], 'diff_bl_stack_cummean': [0.4547696113586426, 0.2001751959323883, nan, nan, nan, nan]}}
30
{'test': {'diff_final_act': [0.05731144919991493, 0.036402083933353424, 0.029847074300050735, nan, nan, nan], 'diff_bl_stack_instant': [0.448456734418869, 0.2982279360294342, 0.25600945949554443, nan, nan, nan], 'diff_bl_stack_cummean': [0.2242283672094345, 0.12304651737213135, 0.09799861907958984, nan, nan, nan]}}
50
{'test': {'diff_final_act': [0.07331066578626633, nan, nan, nan, nan, nan], 'diff_bl_stack_instant': [0.751304566860199, nan, nan, nan, nan, nan], 'diff_bl_stack_cummean': [0.3756522